Imports and Gemini Setup

In [ ]:
import os
import requests
from flask import Flask, request, jsonify
from google import genai
from google.genai import types
from google.api_core import retry
from IPython.display import Markdown, display

# Initialize Flask app
app = Flask(__name__)

# Get API Key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = input("Please enter your Google API key: ")
if not GOOGLE_API_KEY:
    raise ValueError("Google API key not found. Please set the GOOGLE_API_KEY environment variable or provide it manually.")

# Initialize GenAI client
client = genai.Client(api_key=GOOGLE_API_KEY)

# Retry policy for quota or service limit errors
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '_wrapped_'):
    genai.models.Models.generate_content = retry.Retry(predicate=is_retriable)(
        genai.models.Models.generate_content
    )

# Enable grounding via Google Search
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

# Shared memory
conversation_history = []

Gemini Chatbot function for MCP


In [ ]:
def get_gemini_reply(user_input):
    conversation_history.append(types.Content(role="user", parts=[types.Part(text=user_input)]))
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=conversation_history,
        config=config_with_search,
    )
    reply_text = response.candidates[0].content.parts[0].text
    conversation_history.append(types.Content(role="model", parts=[types.Part(text=reply_text)]))
    return reply_text

MCP Webhook Endpoint

In [ ]:
@app.route('/mcp_webhook', methods=['POST'])
def mcp_webhook():
    data = request.json
    user_id = data.get("from")
    user_text = data.get("message", {}).get("text")

    if not user_id or not user_text:
        return jsonify({"error": "Invalid message format"}), 400

    try:
        reply = get_gemini_reply(user_text)
        send_reply_to_mcp(user_id, reply)
        return jsonify({"status": "success"}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

Send Reply Back to MCP

In [ ]:
def send_reply_to_mcp(user_id, text):
    MCP_API_URL = "https://api.gupshup.io/sm/api/v1/msg"  # Replace with your MCP send URL
    MCP_TOKEN = os.getenv("MCP_API_TOKEN") or "Write MCP token here"

    headers = {
        "Authorization": f"Bearer {MCP_TOKEN}",
        "Content-Type": "application/json"
    }

    payload = {
        "to": user_id,
        "message": {"text": text}
    }

    response = requests.post(MCP_API_URL, json=payload, headers=headers)
    if response.status_code != 200:
        print(f"Failed to send MCP message: {response.text}")

CLI Chatbot Loop

In [ ]:
print("Chatbot with memory and search grounding started! Type 'quit' to exit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        print("Exiting the chatbot. Goodbye!")
        break

    conversation_history.append(types.Content(role="user", parts=[types.Part(text=user_input)]))

    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=conversation_history,
            config=config_with_search,
        )
        reply_text = response.candidates[0].content.parts[0].text
        display(Markdown(reply_text))
        conversation_history.append(types.Content(role="model", parts=[types.Part(text=reply_text)]))
    except Exception as e:
        print("An error occurred:", e)

Flask App Run Block

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)